In [92]:
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance

# import time
# import multiprocessing
# from functools import wraps
# import diplib as dip
# from skimage import img_as_ubyte
# from utils.preprocess import resize
# from skimage.filters import threshold_otsu
# from utils.zhangSuenThinning import zhangSuen

In [93]:
print(os.getcwd())

/Users/vishwaksena_dingari/Documents/A Printed Character Recognition System for Meetei-Mayek Script Using Transfer Learning/text-image_to_character-images_segmentation


In [94]:
def addPadding(image, all=None):
    '''
    - ``Input``: an image
    - ``Output``:
        - add padding top: 10, bottom: 20 - ((height + top) % 10), left: 10, right: 20 - ((width + left) % 10)
        - so that the image can be divided into 10 equal parts vertically
    '''
    image = np.asarray(image)
    height, width, channels = image.shape
    # print(height, width, channels)
    image = Image.fromarray(image)
    if all == None:
        top = 10
        bottom = 20 - ((height+top) % 10)
        left = 10
        right = 20 - ((width+left) % 10)
    else: top, right, bottom, left = all, all, all, all
    new_width = width + left + right
    new_height = height + top + bottom
    result = Image.new(image.mode, (new_width, new_height), (255, 255, 255))
    result.paste(image, (left, top))
    result = np.asarray(result)
    # cv2.imshow('result', result)
    # cv2.waitKey(0)
    # print(result.shape)
    return result

In [95]:
def removePadding(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    _, binary_image = cv2.threshold(
        gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV
    )
    for i in range(4):
        while sum(binary_image[0]) == 0:
            image = image[1:]
            binary_image = binary_image[1:]
        image = np.rot90(image)
        binary_image = np.rot90(binary_image)
    return image

In [96]:
def tempDisplay(dict_):
    """
    - `Input`: dictionary
        - ``key``: name for the image to be displayed
        - ``value``: image to be displayed
    - `Output`: a matplotlib plot with the all the images in the dictionary in one image
    """
    rows = int(math.sqrt(len(dict_)))
    cols = math.ceil(len(dict_)/rows)
    fig = plt.figure(figsize=(10, 5))
    i = 1
    for key in dict_:
        fig.add_subplot(rows, cols, i)
        i = i + 1
        plt.imshow(dict_[key])
        plt.axis('off')
        plt.title(str(key))
    plt.show()

In [97]:
def cropImage(pixel_values, original_image, rotate=False):
    '''
    - `Input`: 
        - ``pixel_values``: vertical pixel values range [from, to] to be cropped
        - ``original_image``: image from which the cropped images are needed
        - ``rotate``: if the image is to be rotated or not before the cropping (used for word segmentation form the line images)
    - `Output`: cropped images from the ``original_image``
    '''
    image = original_image
    if rotate:
        image = np.rot90(image, 3)
    cropped_images = []
    h, w = image.shape[:2]
    # dp = {}
    for i in range(len(pixel_values)):
        crop_image = image[pixel_values[i][0]:pixel_values[i][1], 0:w]
        # dp[i] = crop_image
        if rotate:
            cropped_images.append(np.rot90(crop_image))
        else:
            cropped_images.append(crop_image)
    # tempDisplay(dp)
    return cropped_images

In [98]:
# def imageContrast(image):
#     image_contrast = np.zeros(image.shape, image.dtype)
#     alpha = 1.1  # Simple contrast control and alpha value [1.0 - 3.0]
#     beta = 5    # Simple brightness control and beta value [0 - 100]
#     for y in range(image.shape[0]):
#         for x in range(image.shape[1]):
#             for c in range(image.shape[2]):
#                 image_contrast[y, x, c] = np.clip(alpha * image[y, x, c] + beta, 0, 255)
#     return image_contrast
def imageContrast(image, alpha=1.1, beta=5):
    # Convert the input image to a float32 array for better precision
    image = image.astype(np.float32)

    # Apply the contrast and brightness adjustments using NumPy functions
    image_contrast = np.clip(alpha * image + beta, 0, 255).astype(np.uint8)

    image = Image.fromarray(image_contrast)

    new_image = image

    filter = ImageEnhance.Contrast(new_image)
    new_image = filter.enhance(1.25)

    filter = ImageEnhance.Sharpness(new_image)
    new_image = filter.enhance(1.25)
    
    new_image = np.asarray(new_image)
    
    # cv2.imwrite("contrast.jpg", new_image)
    return new_image


In [99]:
def segmentation(image, line_segmentation=False, word_segmentation=False, letter_segmentation=False):
    """
    - `Input`:
        - ``image``: image to be segmented
        - ``rotate``: ``bool`` if the image is to be rotated or not (used for word segmentation form the line images)
    - `Output`: vertical pixel values range [from, to] to be cropped 
    """
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    ret, image = cv2.threshold(
        image, 128, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    
    # ret, image = cv2.threshold(image, 64, 255, cv2.THRESH_BINARY_INV)
    
    # ret, image = cv2.threshold(image, np.mean(image), 255, cv2.THRESH_BINARY_INV)
    
    # rect_kernel_2 = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 4))
    rect_kernel_2 = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 2))
    
    
    if line_segmentation:
        line_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 1))
        image = cv2.erode(image, rect_kernel_2, iterations=1)
        image = cv2.dilate(image, line_kernel, iterations=1)
        # cv2.imwrite("temp.jpg", image)
        
    if word_segmentation:
        image = np.rot90(image, 3)    
        # para_image = cv2.dilate(image, rect_kernel_2, iterations=6)
        image = cv2.dilate(image, rect_kernel_2, iterations=4)
        # cv2.imshow("image", image)
        # cv2.waitKey(0)
        # image = word_image
    
    if letter_segmentation:
        image = np.rot90(image, 3)
        # cv2.imshow("image", image)
        # cv2.waitKey(0)
        # letter_kernel_10 = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
        # letter_kernel_2 = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        # image = cv2.erode(image, letter_kernel_10, iterations=2)
        # image = cv2.dilate(image, letter_kernel_2, iterations=2)
        # image = cv2.dilate(image, rect_kernel_2, iterations=1)
    
    h, w = image.shape[:2]
    pixel_values = []
    temp = []
    for i in range(1, h):
        if len(temp) == 2:
            pixel_values.append(temp)
            temp = []
        
        if len(temp) == 0 and sum(image[i-1]) == 0 and sum(image[i]) != 0:
            temp.append(i - 1)
        
        if len(temp) == 1 and sum(image[i-1]) != 0 and sum(image[i]) == 0:
            temp.append(i + 1)

    return pixel_values

In [100]:
# 83
# img_name = "95.jpg"
# total 1437 not_good 50
# img_name = "zz.jpg"
# total 1257 not_good 32
# img_name = "185.jpg"
# total 216 not_good 23
# img_name = "0_0.jpg"
# total 98 not_good 18
# img_name = "0_1.jpg"
# total 89 not_good 14
# img_name = "0_3.jpg"
# total 83 not_good 5
# img_name = "0_4.jpg"
# total 87 not_good 8
# img_name = "0_5.jpg"
# total 102 not_good 24
img_name = "37.jpg"
# img_name = "212_1.jpg"
IMAGE_PATH = f"{os.getcwd()}/img/{img_name}"
print(IMAGE_PATH)
ORIGINAL_IMAGE = cv2.imread(IMAGE_PATH)
ORIGINAL_IMAGE = addPadding(ORIGINAL_IMAGE)
ORIGINAL_IMAGE = imageContrast(ORIGINAL_IMAGE)



# image = ORIGINAL_IMAGE
# gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
# _, binary_img = cv2.threshold(image, 64, 255, cv2.THRESH_BINARY_INV)
# _, binary_img = cv2.threshold(binary_img, 64, 255, cv2.THRESH_BINARY_INV)


# ORIGINAL_IMAGE = resize(ORIGINAL_IMAGE, 250)

####
# the following function can also be used for line segmentation
# and a similar function can be used for word segmentation
# def line_dilation(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#     ret, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
#     rect_kernel_110 = cv2.getStructuringElement(cv2.MORPH_RECT, (500, 1))
#     dilation = cv2.dilate(thresh, rect_kernel_110, iterations=1)
#     return dilation


/Users/vishwaksena_dingari/Documents/A Printed Character Recognition System for Meetei-Mayek Script Using Transfer Learning/text-image_to_character-images_segmentation/img/37.jpg


In [101]:
temp_path = f"{os.getcwd()}/temp"
folder_names = ["lines", "words", "thin_words", "letters0", "letters1", "letters2"]

if not os.path.exists(temp_path):
    os.mkdir(temp_path)

for folder_name in folder_names:
    if not os.path.exists(f"{temp_path}/{folder_name}"):
        os.mkdir(f"{temp_path}/{folder_name}")

for folder_name in folder_names:
    folder_path = f"{temp_path}/{folder_name}"
    for file in os.listdir(folder_path):
        file_path = f"{folder_path}/{file}"
        if os.path.isfile(file_path):
            os.remove(file_path)

In [102]:
pixel_values = segmentation(ORIGINAL_IMAGE, line_segmentation=True)
ARTICLE_LINES = cropImage(pixel_values, ORIGINAL_IMAGE)

avg_height = 0
for i, line in enumerate(ARTICLE_LINES):
    # print(line.shape[0], end=" ")
    avg_height += line.shape[0]
avg_height = avg_height / len(ARTICLE_LINES)

NEW_ARTICLE_LINES = []
for i, line in enumerate(ARTICLE_LINES):
    if line.shape[0] < avg_height //  2:
        NEW_ARTICLE_LINES[len(NEW_ARTICLE_LINES)-1] = np.vstack((NEW_ARTICLE_LINES[len(NEW_ARTICLE_LINES)-1], line))
    else:
        NEW_ARTICLE_LINES.append(line)
ARTICLE_LINES = NEW_ARTICLE_LINES

for i, line in enumerate(ARTICLE_LINES):
    # tempDisplay({i: ARTICLE_LINES[i]})
    cv2.imwrite(f"{os.getcwd()}/temp/lines/{i}.jpg", line)

In [103]:
ARTICLE_WORDS = []
for line in ARTICLE_LINES:
    pixel_values = segmentation(line, word_segmentation=True)
    words = cropImage(pixel_values, line, rotate=True)
    # words = [resize(word, 2000) for word in words]
    ARTICLE_WORDS.append(words)
    # tempDisplay({i: words[i] for i in range(len(words))})


# mh, mw = 0, 0
# for line in ARTICLE_WORDS:
#     for i, word in enumerate(line):
#         mh, mw = max(mh, word.shape[0]), max(mw, word.shape[1])
# print(mh, mw)

line_count = 0
for line in ARTICLE_WORDS:
    # tempDisplay({i: word for i, word in enumerate(line)})
    for word_count, word in enumerate(line):
        cv2.imwrite(
            f"{os.getcwd()}/temp/words/{line_count}_{word_count}.jpg", word)
    line_count += 1

In [104]:
# ARTICLE_LETTERS = []
# for line in ARTICLE_WORDS:
#     temp_line = []
#     for word in line:
#         pixel_values = segmentation(word, letter_segmentation=True)
#         letters = cropImage(pixel_values, word, True)
#         # tempDisplay({i : letters[i] for i in range(len(letters))})
#         temp_line.append(letters)
#     ARTICLE_LETTERS.append(temp_line)


# for line_count, line in enumerate(ARTICLE_LETTERS):
#     for word_count, word in enumerate(line):
#         # tempDisplay({i: letter for i, letter in enumerate(word)})
#         for letter_count, letter in enumerate(word):
#             cv2.imwrite(f"{os.getcwd()}/temp/letters/{line_count}_{word_count}_{letter_count}.jpg", letter)
        

In [105]:
# cc = 1

def word2Letters(image):
    # image = resize(image, 500)
    org_img = image.copy()
    img_copy = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ret, binary_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # _, binary_img = cv2.threshold(gray, 64, 255, cv2.THRESH_BINARY_INV)
    # _, binary_img = cv2.threshold(gray, np.mean(gray), 255, cv2.THRESH_BINARY_INV)
    
    # dist = cv2.distanceTransform(binary_img, cv2.DIST_C, 3)
    # ret, sure_fg = cv2.threshold(dist, 0.15 * dist.max(), 255, cv2.THRESH_BINARY)
    # sure_fg = sure_fg.astype(np.uint8)
    # binary_img = sure_fg
    
    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    # binary_img = cv2.erode(sure_fg, kernel, iterations=1)
    
    thinned_img = binary_img
    # thinned_img = cv2.ximgproc.thinning(binary_img,  thinningType=cv2.ximgproc.THINNING_GUOHALL)
    # thinned_img = cv2.dilate(thinned_img, kernel, iterations=1)
    
    # thinned_img = zhangSuen(binary_img).astype(np.uint8)
    # thinned_img*=255
    
    # binary_img = np.array(binary_img, dtype=np.uint8)
    # binary_img = binary_img > 128
    # thinned_img = dip.EuclideanSkeleton(binary_img, endPixelCondition='three neighbors')
    # thinned_img = np.array(thinned_img, dtype=np.uint8)
    # thinned_img *= 255
    
    
    # global cc
    # cv2.imwrite(f"{os.getcwd()}/temp/thin_words/{cc}.jpg", thinned_img)
    # cc += 1
    
    contours, hierarchy = cv2.findContours(
        thinned_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = sorted(contours, key=lambda cnt: cv2.boundingRect(cnt)[0])
    cropped_images = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # rect = cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 0, 255), 1)
        x1, y1, x2, y2 = x - 2, y - 2, x + w + 2, y + h + 2
        # x1, y1, x2, y2 = x, y, x + w, y + h
        cropped = org_img[y1 : y2, x1 : x2]
        cropped_images.append(cropped)
    return cropped_images
    
    
    # _, img = cv2.threshold(thinned_img, 75, 255, cv2.THRESH_BINARY_INV)
    # img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    # pixel_values = segmentation(img, letter_segmentation=True)
    # return pixel_values

In [106]:
ARTICLE_LETTERS = []
for line in ARTICLE_WORDS:
    temp_line = []
    for word in line:
        word = addPadding(word, 2)
        # pixel_values = word2Letters(word)
        # letters = cropImage(pixel_values, word, True)
        letters = word2Letters(word)
        temp_line.append(letters)
        # temp_line.append([removePadding(letter) for letter in letters])
    ARTICLE_LETTERS.append(temp_line)


for line_count, line in enumerate(ARTICLE_LETTERS):
    for word_count, word in enumerate(line):
        for letter_count, letter in enumerate(word):
            # letter = removePadding(letter)
            try:
                cv2.imwrite(f"{os.getcwd()}/temp/letters0/{line_count}_{word_count}_{letter_count}.jpg", letter)
            except:
                print("#:", line_count, word_count, letter_count)


width_sum, height_sum, count = 0, 0, 0
for line_count, line in enumerate(ARTICLE_LETTERS):
    for word_count, word in enumerate(line):
        for letter_count, letter in enumerate(word):
            temp_letter = removePadding(letter)
            # temp_letter = letter
            width_sum += temp_letter.shape[1]
            height_sum += temp_letter.shape[0]
            count += 1

In [107]:
AVG_HEIGHT = math.floor(height_sum / count)
AVG_WIDTH = math.floor(width_sum / count)
print(AVG_HEIGHT, AVG_WIDTH)

14 18


In [108]:
def letterSegmentation(letter, avg_width_or_height, rotate = False):
    letter = removePadding(letter)
    AVG_WIDTH_OR_HEIGHT = avg_width_or_height
    sub_let = [letter]
    max_idx = 0     # index of the image with max width or height
    
    while (sub_let[max_idx].shape[1] if rotate else sub_let[max_idx].shape[0]) > AVG_WIDTH_OR_HEIGHT*1.5:
        sub_let_img = sub_let[max_idx]
        
        gray = cv2.cvtColor(sub_let_img, cv2.COLOR_RGB2GRAY)
        
        gray = np.rot90(gray, 3) if rotate else gray
        _, binary_img = cv2.threshold(gray, np.mean(gray), 255, cv2.THRESH_BINARY_INV)
        
        # if rotate: _, binary_img = cv2.threshold(np.rot90(gray, 3), np.mean(gray), 255, cv2.THRESH_BINARY_INV)
        # # _, binary_img = cv2.threshold(np.rot90(gray, 3), 64, 255, cv2.THRESH_BINARY_INV)
        # else: _, binary_img = cv2.threshold(gray, np.mean(gray), 255, cv2.THRESH_BINARY_INV)
        
        pixel_values = []
        sum_histogram = np.asarray([sum(row) for row in binary_img])
        
        if rotate:
            idx_arr = sum_histogram[AVG_WIDTH_OR_HEIGHT//2: len(sum_histogram)-AVG_WIDTH_OR_HEIGHT//4]
        else:
            idx_arr = sum_histogram[AVG_WIDTH_OR_HEIGHT//4:len(sum_histogram)-AVG_WIDTH_OR_HEIGHT//4]
        
        if idx_arr.size == 0: break
        
        # idx = AVG_WIDTH_OR_HEIGHT // 2 + idx_arr.index(min(idx_arr))
        if rotate: idx =  AVG_WIDTH_OR_HEIGHT//2 + np.where(idx_arr==np.min(idx_arr[np.nonzero(idx_arr)]))[0][0]
        else: idx = AVG_WIDTH_OR_HEIGHT//4 + np.where(idx_arr==np.min(idx_arr[np.nonzero(idx_arr)]))[0][0]
        
        
        left, right = sub_let[:max_idx], sub_let[max_idx+1:]
        # pixel_values = [[0, idx+1], [idx-1, len(binary_img)]]
        if rotate: pixel_values = [[0, idx], [idx, len(binary_img)]]
        else: pixel_values = [[0, idx+1], [idx-1, len(binary_img)]]
        
        if rotate: cropped_sub_let = cropImage(pixel_values, sub_let_img, rotate=True)
        else: cropped_sub_let = cropImage(pixel_values, sub_let_img)
        
        sub_let = [*left, *cropped_sub_let, *right]
        
        # max_idx = sub_let.index(max(sub_let, key=lambda x: x.shape[1]))
        max_idx = 0
        for i, img in enumerate(sub_let):
            if rotate:
                if img.shape[1] > sub_let[max_idx].shape[1]:
                    max_idx = i
            else:
                if img.shape[0] > sub_let[max_idx].shape[0]:
                    max_idx = i
        # for i, img in enumerate(sub_let):
        #     if img == []
    return sub_let

In [109]:
TEMP_ARTICLE_LETTERS = []
for line_count, line in enumerate(ARTICLE_LETTERS):
    temp_line = []
    for word_count, word in enumerate(line):
        temp_word = []
        for letter_count, letter in enumerate(word):
            if letter.shape[1] > AVG_WIDTH*1.25:
                # print("w", line_count, word_count, letter_count)
                sub_let_horizontal = letterSegmentation(letter, AVG_WIDTH, True)
                temp_word = [*temp_word, *sub_let_horizontal]
            else: temp_word.append(letter)
        temp_line.append(temp_word)
    TEMP_ARTICLE_LETTERS.append(temp_line)
ARTICLE_LETTERS = TEMP_ARTICLE_LETTERS

for line_count, line in enumerate(ARTICLE_LETTERS):
    for word_count, word in enumerate(line):
        for letter_count, letter in enumerate(word):
            try:
                cv2.imwrite(f"{os.getcwd()}/temp/letters1/{line_count}_{word_count}_{letter_count}.jpg", letter)
                # cv2.imwrite(f"{os.getcwd()}/temp/letters1/{line_count}_{word_count}_{letter_count}.jpg", imageContrast(cv2.resize(letter, (32, 32))))
            except:
                print("#:", line_count, word_count, letter_count)

TEMP_ARTICLE_LETTERS = []
for line_count, line in enumerate(ARTICLE_LETTERS):
    temp_line = []
    for word_count, word in enumerate(line):
        temp_word = []
        for letter_count, letter in enumerate(word):
            # if line_count == 6 and word_count == 1 and letter_count == 0: continue
            # if line_count == 11 and word_count == 3 and letter_count == 0: continue
            if letter.shape[0] > AVG_HEIGHT*1.25:
                # print("h", line_count, word_count, letter_count)
                # sub_let_vertical = []
                # print(len(sub_let_vertical), end="####")
                # process = multiprocessing.Process(target=letterSegmentation, args=[letter, AVG_HEIGHT, False, sub_let_vertical])
                # process.start()
                
                # if process.is_alive():
                #     process.kill()
                #     cv2.imwrite(
                #         f"./not_working/{line_count}_{word_count}_{letter_count}.jpg", letter)
                #     process.join()
                sub_let_vertical = letterSegmentation(letter, AVG_HEIGHT)
                # print(len(sub_let_vertical))
                temp_word = [*temp_word, *sub_let_vertical]
            else:
                temp_word.append(letter)
        temp_line.append(temp_word)
    TEMP_ARTICLE_LETTERS.append(temp_line)
ARTICLE_LETTERS = TEMP_ARTICLE_LETTERS

for line_count, line in enumerate(ARTICLE_LETTERS):
    for word_count, word in enumerate(line):
        for letter_count, letter in enumerate(word):
            try:
                # cv2.imwrite(f"{os.getcwd()}/temp/letters2/{line_count}_{word_count}_{letter_count}.jpg", letter)
                cv2.imwrite(f"{os.getcwd()}/temp/letters2/{line_count}_{word_count}_{letter_count}.jpg", imageContrast(cv2.resize(letter, (32, 32))))
            except:
                print("#:", line_count, word_count, letter_count)